In [17]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV , train_test_split , cross_val_score
from sklearn.metrics import accuracy_score 
import warnings

In [18]:
train = pd.read_csv('./data/train.csv' ,header=None)
test = pd.read_csv('./data/test.csv' , header=None)
label = pd.read_csv('./data/trainLabels.csv' , header=None)

In [19]:
X_train , X_test , y_train , y_test = train_test_split(train , label , train_size=0.8 , random_state=1)

In [20]:
model = GradientBoostingClassifier()
model.fit(X_train , y_train.values.ravel())
y_pred = model.predict(X_test)
accuracy_score(y_test , y_pred)

0.925

In [21]:
#MinMax
from sklearn.preprocessing import MinMaxScaler
mn = MinMaxScaler()
X_train_mn = mn.fit_transform(X_train)
X_test_mn = mn.transform(X_test)

model.fit(X_train_mn , y_train)
y_pred = model.predict(X_test_mn)
accuracy_score(y_test , y_pred)

c:\Users\louisan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.925

In [22]:
model = GradientBoostingClassifier()
n_estimators = [i for i in range(100 , 200 , 10)]
learing = [0.1 , 0.2 , 0.3 ,0.4 , 0.5 , 0.8 , 0.05 , 0.08 , 0.01]
param_grid = dict(n_estimators=n_estimators , learning_rate=learing)
GSV = GridSearchCV(model , param_grid , scoring='accuracy' , n_jobs=-1 , verbose=1)
Gsv_result = GSV.fit(X_train , y_train.values.ravel())
print(f'score:{Gsv_result.best_score_} , using:{Gsv_result.best_params_}')

Fitting 5 folds for each of 90 candidates, totalling 450 fits
score:0.865 , using:{'learning_rate': 0.4, 'n_estimators': 140}


In [23]:
new_model = GradientBoostingClassifier(n_estimators=Gsv_result.best_params_['n_estimators'] ,
                                      learning_rate=Gsv_result.best_params_['learning_rate'])
new_model.fit(X_train , y_train.values.ravel())
y_pred = new_model.predict(X_test)
accuracy_score(y_test , y_pred)

0.885

In [24]:
cross_val_score(new_model , train , label.values.ravel() , cv=5).mean()

0.867

In [25]:
test_result = new_model.predict(test)
result = pd.DataFrame(test_result)
result

,0
0,1
1,0
2,0
3,0
4,0
...,...
8995,1
8996,1
8997,1
8998,0


In [26]:
result = result.reset_index().rename(columns={'index':'Id' , 0:'Solution'})
result['Id'] = result['Id'] +1
result

,Id,Solution
0,1,1
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
8995,8996,1
8996,8997,1
8997,8998,1
8998,8999,0


In [27]:
result.to_csv('./Scikit-learn_test.csv',index=False)